# Homework 3: Vector Search

In this homework, we'll experiemnt with vector with and without Elasticsearch

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Getting the embeddings model

First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```python
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```python
user_question = "I just discovered the course. Can I still join it?"
```

What's the first value of the resulting vector?

* -0.24
* -0.04
* **0.07** (answer)
* 0.27

In [19]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

user_question = "I just discovered the course. Can I still join it?"
encoded_user_question = embedding_model.encode(user_question)

print(encoded_user_question[0])

0.07822264


## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

```python
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
```

We will use only a subset of the questions - the questions
for `"machine-learning-zoomcamp"`. After filtering, you should
have only 375 documents


In [20]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()


documents = [ item for item in documents if item["course"] == "machine-learning-zoomcamp"]


In [21]:
print(len(documents))

375


In [22]:
import json

print(json.dumps(documents[0], indent=4))

{
    "text": "Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there\u2019s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork",
    "section": "General course-related questions",
    "question": "How do I sign up?",
    "course": "machine-learning-zoomcamp",
    "id": "0227b872"
}


## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses. 

**(375, 768)** (answer)

In [23]:
from tqdm import tqdm
import numpy as np

In [24]:
embeddings_list = []

for doc in tqdm(documents):
    question = doc['question']
    text     = doc['text']
    qa_text = f'{question} {text}'
    embeddings = embedding_model.encode(qa_text)
    embeddings_list.append(embeddings)

embeddings_array = np.array(embeddings_list)

100%|██████████| 375/375 [00:52<00:00,  7.12it/s]


In [25]:
print(embeddings_array.shape)

(375, 768)


## Running ElasticSearch:

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [26]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': '7c005c6f76ea', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'I-stArAVTj-2-gpJFaIPUw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return something very close to 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:


```python
scores = X.dot(v)
```

What's the highest score in the results?

- 65.0 
- 6.5
- **0.65** (answer)
- 0.065

In [27]:
scores = embeddings_array.dot(encoded_user_question)
max_score = np.max(scores)
print(max_score)

0.6506573


## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)
```

If you don't understand how the `search` function work:

* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace `argsort` with `argpartition` to make it a lot faster)

In [28]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10, mode="argsort"):
        scores = self.embeddings.dot(v_query)
        if mode == "argsort":
            idx = np.argsort(-scores)[:num_results]
        elif mode == "argpartition":
            idx = np.argpartition(-scores)[:num_results]
        else:
            raise ValueError("Invalid mode. Mode must be 'argsort' or 'argpartition'")
        return [self.documents[i] for i in idx]



In [29]:
search_engine = VectorSearchEngine(documents=documents, embeddings=embeddings_array)
search_engine.search(encoded_user_question, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of
`VectorSearchEngine` with `num_results=5`.

What did you get?

* **0.93** (answer)
* 0.73
* 0.53
* 0.33

In [30]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [31]:
df_ground_truth.head(5)

,question,course,document
2123,Where can I sign up for the course?,machine-learning-zoomcamp,0227b872
2124,Can you provide a link to sign up?,machine-learning-zoomcamp,0227b872
2125,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0227b872
2126,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0227b872
2127,How can I structure my questions and answers f...,machine-learning-zoomcamp,0227b872


In [32]:
relevance_total = []

for gt in tqdm(ground_truth):
    doc_id = gt["document"]
    gt_question = gt["question"]
    gt_embedding_question = embedding_model.encode(gt_question)
    search_results = search_engine.search(gt_embedding_question, num_results=5)
    relevance = [ d['id'] == doc_id for d in search_results ]
    relevance_total.append(relevance)

100%|██████████| 1830/1830 [04:47<00:00,  6.37it/s]


In [33]:
def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relevance_total)

In [34]:
relevance_total_array = np.array(relevance_total)
num_gt_items, num_results = relevance_total_array.shape

In [35]:
recall = np.mean(np.sum(relevance_total_array, axis=1))
print("hit rate (recall): ", recall, hit_rate(relevance_total_array))

hit rate (recall):  0.9426229508196722 0.9398907103825137


In [36]:
coeff_array = 1 / np.arange(1, 1 + num_results)

mmr = np.mean(np.sum(np.multiply(coeff_array, relevance_total_array) , axis=1))
print("mmr:", mmr)


mmr: 0.851648451730419


## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

id: **636f55d5**
score: **0.7375**

In [37]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [38]:
# clean and initialize index
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [39]:
#created the dense vector using the pre-trained model
operations = []
for doc in tqdm(documents):
    # Transforming the title into an embedding using the model
    doc["text_vector"] = embedding_model.encode(doc["text"]).tolist()
    operations.append(doc)

100%|██████████| 375/375 [00:48<00:00,  7.77it/s]


In [40]:
print(json.dumps(operations[0],indent=4))

{
    "text": "Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there\u2019s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork",
    "section": "General course-related questions",
    "question": "How do I sign up?",
    "course": "machine-learning-zoomcamp",
    "id": "0227b872",
    "text_vector": [
        0.01943064294755459,
        -0.003654015250504017,
        0.04042726755142212,
        0.06813299655914307,
        0.07546191662549973,
        -0.06538787484169006,
        -0.07440777122974396,
        0.04882427677512169,
        0.012547491118311882,
        -0.003298628143966198,
        -0.013346671126782894,
        -0.036168426275253296,
        0.02879025973379612,
        -0.0

In [41]:
for doc in tqdm(operations):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|██████████| 375/375 [00:03<00:00, 110.30it/s]


In [42]:
k_value = 5

query = {
    "field": "text_vector",
    "query_vector": encoded_user_question,
    "k": k_value,
    "num_candidates": 10000, 
}

res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])

In [43]:
print(json.dumps(res["hits"]["hits"][0], indent=4))

{
    "_index": "course-questions",
    "_id": "HFN7uZABtRxsK5XrC743",
    "_score": 0.73748606,
    "_source": {
        "question": "When does the next iteration start?",
        "course": "machine-learning-zoomcamp",
        "section": "General course-related questions",
        "text": "The course is available in the self-paced mode too, so you can go through the materials at any time. But if you want to do it as a cohort with other students, the next iterations will happen in September 2023, September 2024 (and potentially other Septembers as well)."
    }
}


In [44]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [45]:
high_id = generate_document_id(res["hits"]["hits"][0]["_source"])
print(high_id)

636f55d5


## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between
the query and ALL the vectors in our database. Usually this is 
not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster. 

Let's evaluate how worse the results are when we switch from
exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93
* **0.73** (answer)
* 0.53
* 0.33


In [46]:
relevance_total_es = []

for gt in tqdm(ground_truth):
    doc_id = gt["document"]
    gt_question = gt["question"]
    gt_embedding_question = embedding_model.encode(gt_question)
    query = {
        "field": "text_vector",
        "query_vector": gt_embedding_question,
        "k": k_value,
        "num_candidates": 10000, 
    }
    res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
    # print(gt_question)
    result_hash_codes = [ generate_document_id(item["_source"]) for item in res["hits"]["hits"] ]
    relevance = [ doc_id == item for item in result_hash_codes]
    # print(json.dumps(res['hits']['hits'], indent=4))
    # print(doc_id)
    # print(result_hash_codes)
    # print(relevance)
    relevance_total_es.append(relevance)

100%|██████████| 1830/1830 [01:26<00:00, 21.11it/s]


In [47]:
relevance_total_array_es = np.array(relevance_total_es)
num_gt_items, num_results = relevance_total_array_es.shape
print(num_gt_items, num_results)

1830 5


In [48]:
recall = np.mean(np.sum(relevance_total_array_es, axis=1))
print("hit rate (recall): ", recall, hit_rate(relevance_total_array_es))

hit rate (recall):  0.8808743169398907 0.8803278688524591


In [50]:
print(np.shape(np.sum(relevance_total_array_es, axis=1)))

(1830,)


In [49]:
coeff_array = 1 / np.arange(1, 1 + k_value)

mmr = np.mean(np.sum(np.multiply(coeff_array, relevance_total_array_es) , axis=1))
print("mmr:", mmr)


mmr: 0.7658925318761384


## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw3
* It's possible that your answers won't match exactly. If it's the case, select the closest one.